In [1]:
from datasets import load_dataset, list_datasets
import transformers
import torch

In [2]:
books = load_dataset('bookcorpus',split='train')

Reusing dataset bookcorpus (/mounts/data/corp/huggingface/datasets/bookcorpus/plain_text/1.0.0/44662c4a114441c35200992bea923b170e6f13f2f0beb7c14e43759cec498700)


In [3]:
model = transformers.BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
tokenizer = transformers.BertTokenizerFast.from_pretrained('bert-base-uncased')

In [5]:
mini = books.select(range(1000)) # small slice of dataset for testing

In [6]:
tokenized = mini.map(lambda e: tokenizer(e['text'],truncation=True,padding='max_length'),remove_columns=['text']) # remember to add padding to have all tokenizations of same length

Loading cached processed dataset at /mounts/data/corp/huggingface/datasets/bookcorpus/plain_text/1.0.0/44662c4a114441c35200992bea923b170e6f13f2f0beb7c14e43759cec498700/cache-32c445a4783b1cce.arrow


In [7]:
tokenized.set_format('torch') #set tensors to pytorch format. Remember to drop 'text' column before you do this

In [8]:
dataloader = torch.utils.data.DataLoader(tokenized,batch_size=8)

In [9]:
batch = next(iter(dataloader))

In [10]:
output = model(**batch)

In [11]:
output.last_hidden_state.shape

torch.Size([8, 512, 768])

In [12]:
sentence_embedding = output.last_hidden_state.mean(1) #taking the mean of tokens to get sentence embedding

In [13]:
sentence_embedding.shape

torch.Size([8, 768])